In [46]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [115]:
df = pd.read_excel('Assignment 5.xls', index_col='User')
ndf = pd.read_excel('Assignment 5.xls', index_col='User', sheetname='NormRatings')

In [116]:
df.drop('Mean', inplace=True, axis=1) # remove coluna Mean
l2_norm = df.loc['L2'] # salva l2 norm
df.drop('L2', inplace=True) # remove row l2 norm

ndf.drop('L2', inplace=True) # remove row l2 norm

    Similarity between Toy Story and Star Wars, raw: 0.645
    Similarity between Toy Story and Star Wars, normalized: -0.378

In [52]:
df_cos = cosine_similarity(df.T.fillna(0), df.T.fillna(0))
dfcos = pd.DataFrame(df_cos, index=df.columns, columns=df.columns)

dfcos['1: Toy Story (1995)']

1: Toy Story (1995)                                        1.000000
1210: Star Wars: Episode VI - Return of the Jedi (1983)    0.644995
356: Forrest Gump (1994)                                   0.580540
318: Shawshank Redemption, The (1994)                      0.667424
593: Silence of the Lambs, The (1991)                      0.570229
3578: Gladiator (2000)                                     0.587852
260: Star Wars: Episode IV - A New Hope (1977)             0.747409
2028: Saving Private Ryan (1998)                           0.534579
296: Pulp Fiction (1994)                                   0.667846
1259: Stand by Me (1986)                                   0.492659
2396: Shakespeare in Love (1998)                           0.376659
2916: Total Recall (1990)                                  0.623056
780: Independence Day (ID4) (1996)                         0.690665
541: Blade Runner (1982)                                   0.383067
1265: Groundhog Day (1993)                      

In [53]:
ndf_cos = cosine_similarity(ndf.T.fillna(0), ndf.T.fillna(0))
ndfcos = pd.DataFrame(ndf_cos, index=ndf.columns, columns=ndf.columns)

ndfcos['1: Toy Story (1995)']

1: Toy Story (1995)                                        1.000000
1210: Star Wars: Episode VI - Return of the Jedi (1983)   -0.377771
356: Forrest Gump (1994)                                   0.355780
318: Shawshank Redemption, The (1994)                      0.215975
593: Silence of the Lambs, The (1991)                     -0.603708
3578: Gladiator (2000)                                     0.139214
260: Star Wars: Episode IV - A New Hope (1977)            -0.050338
2028: Saving Private Ryan (1998)                           0.192799
296: Pulp Fiction (1994)                                   0.295013
1259: Stand by Me (1986)                                  -0.225657
2396: Shakespeare in Love (1998)                           0.116755
2916: Total Recall (1990)                                  0.184712
780: Independence Day (ID4) (1996)                        -0.319904
541: Blade Runner (1982)                                   0.020007
1265: Groundhog Day (1993)                      

# Top 5 Toy Story

## Unnormalized

In [55]:
dfcos['1: Toy Story (1995)'].nlargest(6)

1: Toy Story (1995)                               1.000000
260: Star Wars: Episode IV - A New Hope (1977)    0.747409
780: Independence Day (ID4) (1996)                0.690665
296: Pulp Fiction (1994)                          0.667846
318: Shawshank Redemption, The (1994)             0.667424
1265: Groundhog Day (1993)                        0.661016
Name: 1: Toy Story (1995), dtype: float64

## Normalized

In [56]:
ndfcos['1: Toy Story (1995)'].nlargest(6)

1: Toy Story (1995)                      1.000000
34: Babe (1995)                          0.554448
356: Forrest Gump (1994)                 0.355780
296: Pulp Fiction (1994)                 0.295013
318: Shawshank Redemption, The (1994)    0.215975
2028: Saving Private Ryan (1998)         0.192799
Name: 1: Toy Story (1995), dtype: float64

# Top 5 for User 5277

In [58]:
df.loc[5277]

1: Toy Story (1995)                                        1.0
1210: Star Wars: Episode VI - Return of the Jedi (1983)    NaN
356: Forrest Gump (1994)                                   NaN
318: Shawshank Redemption, The (1994)                      2.0
593: Silence of the Lambs, The (1991)                      4.0
3578: Gladiator (2000)                                     2.0
260: Star Wars: Episode IV - A New Hope (1977)             5.0
2028: Saving Private Ryan (1998)                           NaN
296: Pulp Fiction (1994)                                   NaN
1259: Stand by Me (1986)                                   4.0
2396: Shakespeare in Love (1998)                           3.0
2916: Total Recall (1990)                                  2.0
780: Independence Day (ID4) (1996)                         2.0
541: Blade Runner (1982)                                   NaN
1265: Groundhog Day (1993)                                 2.0
2571: Matrix, The (1999)                               

In [ ]:
nota do usuário * similaridade dos items / sum(similaridades)

## Unnormalized

In [ ]:
def predict_user_n_norm(user, n_neighbor):
    
    neighbor_corr = top_n_corr(user, n_neighbor, exclude_user=True)
    
    # numerator
    # rating - média do rating p/ o user
    rn_ = df.subtract(df.mean())
    
    numerator = rn_.fillna(0).multiply(neighbor_corr).sum(axis=1)
    
    # numerator
    denominator = (~df.isnull()).multiply(neighbor_corr).sum(axis=1)
    
    return df.loc[:, user].mean() + numerator / denominator

In [142]:
numerator = df.loc[5277].fillna(0).dot(dfcos)
denominator = df.loc[5277].notnull().dot(dfcos)

(numerator / denominator).nlargest(5)

527: Schindler's List (1993)                      2.973883
1259: Stand by Me (1986)                          2.928801
260: Star Wars: Episode IV - A New Hope (1977)    2.922240
593: Silence of the Lambs, The (1991)             2.883304
2396: Shakespeare in Love (1998)                  2.852131
Name: 5277, dtype: float64

## Normalized

In [71]:
nndfcos = ndfcos.where(ndfcos > 0, 0) # remove similaridades negativas

    What is the ID of the third highest recommended movie for user 5277?

    1259: Stand by Me (1986)  

In [141]:
numerator = ndf.loc[5277].dot(nndfcos)
denominator = df.loc[5277].notnull().dot(nndfcos)

(numerator / denominator).nlargest(5)

260: Star Wars: Episode IV - A New Hope (1977)    1.796591
527: Schindler's List (1993)                      1.793252
1259: Stand by Me (1986)                          0.832911
2396: Shakespeare in Love (1998)                  0.528708
593: Silence of the Lambs, The (1991)             0.451659
Name: 5277, dtype: float64